## Package Imports

In [2]:
import pandas as pd
import requests
import json
from sklearn.model_selection import train_test_split

import xplainable as xp
from xplainable.core.models import XClassifier
from xplainable.core.optimisation.bayesian import XParamOptimiser
from xplainable.preprocessing.pipeline import XPipeline
from xplainable.preprocessing import transformers as xtf

import xplainable_client

## Read in the csv dataset

In [4]:
df = pd.read_csv("data/asset_failure.csv")

In [5]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


## Dataset Overview: Machine Failure Prediction

This dataset is designed for predictive maintenance, focusing on machine failure prediction. Below is an overview of its structure and the data it contains:

1. **UDI (Unique Identifier):** A column for unique identification numbers for each record.

2. **Product ID:** Identifier for the product being produced or involved in the process.

3. **Type:** Indicates the type or category of the product or process, with different types represented by different letters (e.g., 'M', 'L').

4. **Air temperature [K] (Kelvin):** The temperature of the air in the environment where the machine operates, measured in Kelvin.

5. **Process temperature [K] (Kelvin):** The operational temperature of the process or machine, also measured in Kelvin.

6. **Rotational speed [rpm] (Revolutions per Minute):** This column shows the speed at which a component of the machine is rotating.

7. **Torque [Nm] (Newton Meters):** The torque being applied in the process, measured in Newton meters.

8. **Tool wear [min]:** Indicates the amount of wear on the tools used in the machine, measured in minutes of operation.

9. **Machine failure:** A binary indicator (0 or 1) showing whether a machine failure occurred.

10. **TWF (Tool Wear Failure):** Specific indicator of failure due to tool wear.

11. **HDF (Heat Dissipation Failure):** Indicates failure due to ineffective heat dissipation.

12. **PWF (Power Failure):** Shows whether a failure was due to power issues.

13. **OSF (Overstrain Failure):** Indicates if the failure was due to overstraining of the machine components.

14. **RNF (Random Failure):** A column for failures that don't fit into the other specified categories and are considered random.

Each row of the dataset represents a unique instance or record of the production process, with the corresponding measurements and failure indicators. This data can be used to train machine learning models to predict machine failures based on these parameters.


In [6]:
df = df.drop(columns=["Product ID", "UDI", "TWF", "HDF", "PWF", "OSF", "RNF"])
df

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure
0,M,298.1,308.6,1551,42.8,0,0
1,L,298.2,308.7,1408,46.3,3,0
2,L,298.1,308.5,1498,49.4,5,0
3,L,298.2,308.6,1433,39.5,7,0
4,L,298.2,308.7,1408,40.0,9,0
...,...,...,...,...,...,...,...
9995,M,298.8,308.4,1604,29.5,14,0
9996,H,298.9,308.4,1632,31.8,17,0
9997,M,299.0,308.6,1645,33.4,22,0
9998,H,299.0,308.7,1408,48.5,25,0


In [7]:
df["Machine failure"].value_counts()

0    9661
1     339
Name: Machine failure, dtype: int64

In [8]:
X, y = df.drop(columns=['Machine failure']), df['Machine failure']

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)

## 2. Model Optimisation
The `XParamOptimiser` is utilised to fine-tune the hyperparameters of our model. This process searches for the optimal parameters that will yield the best model performance, balancing accuracy and computational efficiency.

In [9]:
opt = XParamOptimiser()
params = opt.optimise(X_train, y_train)

100%|██████████| 30/30 [00:01<00:00, 17.32trial/s, best loss: -0.9315957452699021]


## 3. Model Training
With the optimised parameters obtained, the `XClassifier` is trained on the dataset. This classifier undergoes a fitting process with the training data, ensuring that it learns the underlying patterns and can make accurate predictions.

In [10]:
model = XClassifier(**params)
model.fit(X_train, y_train)

## 4. Model Interpretability and Explainability
Following training, the `model.explain()` method is called to generate insights into the model's decision-making process. This step is crucial for understanding the factors that influence the model's predictions and ensuring that the model's behaviour is transparent and explainable.

In [11]:
model.explain()

alt.HConcatChart(...)

In [18]:
params = {
    "max_depth": 4,
    "min_info_gain": 0.05,
}

model.update_feature_params(features=['Rotational speed [rpm]', 'Tool wear [min]', 'Air temperature [K]', 'Process temperature [K]','Torque [Nm]'], **params)

In [19]:
model.explain()

alt.HConcatChart(...)

In this snapshot, we demonstrate the impact of hyperparameter tuning on model interpretability. By adjusting max_depth and min_info_gain, we refine the feature wise explainability and information criterion, respectively, which in turn recalibrates feature score contributions. These scores, essential in understanding feature contributions to model predictions, are visualized before and after parameter adjustment, illustrating the model's internal logic shifts. This process is critical for enhancing transparency and aids in pinpointing influential features, fostering the development of interpretable and trustworthy machine learning models.

# Persisting to Xplainable Cloud

## Instantiate Xplainable Cloud

Initialise the xplainable cloud using an API key from:
https://beta.xplainable.io/

This allows you to save and collaborate on models, create deployments, create shareable reports.

In [20]:
client = xplainable_client.Client(
    api_key="",#<- Add your own token here
)

## 5. Model Persisting
In this step, we first create a unique identifier for our churn prediction model using `client.create_model_id`. This identifier, shown as `model_id`, represents the newly instantiated model which predicts the likelihood of customers leaving within the next month. Following this, we generate a specific version of the model with `xp.client.create_model_version`, passing in our training data. The output `version_id` represents this particular iteration of our model, allowing us to track and manage different versions systematically.


In [22]:
# Create a model
model_id = client.create_model(
    model=model,
    model_name="Asset Failure Prediction",
    model_description="Using machine metadata to predict asset failures",
    x=X_train,
    y=y_train
)

  0%|          | 0/6 [00:00<?, ?it/s]

<Response [200]>


## 6. Model Deployment

The code block illustrates the deployment of our churn prediction model using the `xp.client.deploy` function. The deployment process involves specifying the `hostname` of the server where the model will be hosted, as well as the unique `model_id` and `version_id` that we obtained in the previous steps. This step effectively activates the model's endpoint, allowing it to receive and process prediction requests. The output confirms the deployment with a `deployment_id`, indicating the model's current `status` as 'inactive', its `location`, and the `endpoint` URL where it can be accessed for xplainable deployments.

In [24]:
deployment = client.deploy(
    model_version_id=model_id["version_id"], #<- Use model id produced above
)

<Response [200]>


### Testing the Deployment programatically
This section demonstrates the steps taken to programmatically test a deployed model. These steps are essential for validating that the model's deployment is functional and ready to process incoming prediction requests.


1. **Activating the Deployment:**
   The model deployment is activated using `xp.client.activate_deployment`, which changes the deployment status to active, allowing it to accept prediction requests.

In [25]:
client.activate_deployment(deployment['deployment_id'])

{'message': 'activated deployment'}

2. **Creating a Deployment Key:**
   A deployment key is generated with `xp.client.generate_deploy_key`. This key is required to authenticate and make secure requests to the deployed model.

In [27]:
deploy_key = client.generate_deploy_key(deployment['deployment_id'],'API key for Tool Failure Prediction', 7)

<Response [200]>


3. **Generating Example Payload:**
   An example payload for a deployment request is generated by `xp.client.generate_example_deployment_payload`. This payload mimics the input data structure the model expects when making predictions.

In [29]:
#Set the option to highlight multiple ways of creating data
option = 2

In [31]:
if option == 1:
    body = xp.client.generate_example_deployment_payload(deployment['deployment_id'])
else:
    body = json.loads(df.drop(columns=["Machine failure"]).sample(1).to_json(orient="records"))

In [32]:
body

[{'Type': 'L',
  'Air temperature [K]': 297.9,
  'Process temperature [K]': 307.8,
  'Rotational speed [rpm]': 1397,
  'Torque [Nm]': 45.0,
  'Tool wear [min]': 104}]

4. **Making a Prediction Request:**
   A POST request is made to the model's prediction endpoint with the example payload. The model processes the input data and returns a prediction response, which includes the predicted class (e.g., 0 for no failure) and the prediction probabilities for each class.

In [33]:
response = requests.post(
    url="https://inference.xplainable.io/v1/predict",
    headers={'api_key': deploy_key['deploy_key']},
    json=body
)

value = response.json()
value

[{'index': 0,
  'id': None,
  'partition': '__dataset__',
  'score': 0.00400689018470244,
  'proba': None,
  'pred': 0,
  'support': None,
  'breakdown': [{'feature': 'base_value',
    'value': None,
    'score': 0.035522388059701496},
   {'feature': 'Type', 'value': 'L', 'score': 0.0},
   {'feature': 'Air temperature [K]', 'value': '297.9', 'score': 0.0},
   {'feature': 'Process temperature [K]', 'value': '307.8', 'score': 0.0},
   {'feature': 'Rotational speed [rpm]',
    'value': '1397',
    'score': -0.013910922832375576},
   {'feature': 'Torque [Nm]', 'value': '45', 'score': -0.011265812711409608},
   {'feature': 'Tool wear [min]',
    'value': '104',
    'score': -0.0063387623312138736}]}]